## Lista 1 ex6

### Primeiro, vamos importar as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [45]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import math

### Inicializando janela

In [46]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Cores", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)


### GLSL para Vertex Shader


In [47]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

### GLSL para Fragment Shader


### Possibilitando modificar a cor.


In [48]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [49]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


### Associando nosso código-fonte aos slots solicitados

In [50]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [51]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")


### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [52]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [53]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


### Linkagem do programa

In [54]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, nós compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.


In [55]:
vertices = np.zeros(3, [("position", np.float32, 2)])

In [56]:
vertices['position'] = [
                            ( 0.0, +1.0), # vertice 0
                            (+1.0, 0.0), # vertice 1
                            (+0.0, +0.0) # vertice 2
                        ]

### Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [57]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)


### Abaixo, nós enviamos todo o conteúdo da variável vertices.

Veja os parâmetros da função glBufferData [https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glBufferData.xhtml]

In [58]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

### Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

Primeiro, definimos o byte inicial e o offset dos dados.

In [59]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)


Em seguida, soliciamos à GPU a localização da variável "position" (que guarda coordenadas dos nossos vértices). Nós definimos essa variável no Vertex Shader.

In [60]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

A partir da localização anterior, nós indicamos à GPU onde está o conteúdo (via posições stride/offset) para a variável position (aqui identificada na posição loc).

Outros parâmetros:

* Definimos que possui duas coordenadas
* Que cada coordenada é do tipo float (GL_FLOAT)
* Que não se deve normalizar a coordenada (False)

Mais detalhes: https://www.khronos.org/registry/OpenGL-Refpages/gl4/html/glVertexAttribPointer.xhtml

In [61]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

###  Novidade aqui! Vamos pegar a localização da variável color (uniform) para que possamos alterá-la em nosso laço da janela!

In [62]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0

### Capturando eventos de teclado e modificando variáveis para a matriz de transformação

In [63]:
t_x = 0
t_y = 0

s_x = 1
s_y = 1

angulo = 0
def key_event(window,key,scancode,action,mods):
    global t_y, t_x, s_x, s_y, angulo
    
    #if key == 265: #cima
    #    t_y += 0.01 
    #    s_y += 0.01
        
    #if key == 264: #baixo
    #    t_y -= 0.01 
    #    s_y -= 0.01
        
    #if key == 263: #esquerda
    #    t_x -= 0.01 
    #    s_x -= 0.01
    #    angulo += 0.01 
        
    if key == 262: #direita
        t_x += 3.0 
        t_y += -2.0
        s_x += 1.0 
        s_y += 1.0
        angulo += 53.13 
    
glfw.set_key_callback(window,key_event)

### Multiplicação de matrizes

In [64]:
def mult(m1, m2):
    m1 = np.reshape(m1, (4,4))
    m2 = np.reshape(m2, (4,4))
    
    M = np.matmul(m1, m2)
    mat_final = np.reshape(M, (1, 16))
    
    return mat_final

### Nesse momento, nós exibimos a janela!


In [65]:
glfw.show_window(window)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

A novidade agora é a função glDrawArrays()

Tal função recebe o tipo de primitiva (GL_TRIANGLES), o índice inicial do array de vértices (vamos mostrar todos os três vértices, por isso começamos com 0) e a quantidade de vértices ( len(vertices) ).

In [66]:

while not glfw.window_should_close(window):

    glfw.poll_events() 

    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    
    #Draw Square
    mat_translation = np.array([  1.0, 0.0, 0.0, t_x, 
                                  0.0, 1.0, 0.0, t_y, 
                                  0.0, 0.0, 1.0, 0.0, 
                                  0.0, 0.0, 0.0, 1.0], np.float32)
    
    c = math.cos(angulo)
    s = math.sin(angulo)
    mat_rotation = np.array([   c, -s, 0.0, 0.0, 
                                s, c, 0.0, 0.0, 
                                0.0, 0.0, 1.0, 0.0, 
                                0.0, 0.0, 0.0, 1.0], np.float32)


    mat_scale = np.array([  s_x, 0.0, 0.0, 0.0, 
                            0.0, s_x, 0.0, 0.0, 
                            0.0, 0.0, 1.0, 0.0, 
                            0.0, 0.0, 0.0, 1.0], np.float32)
    
    
    mat_shearing = np.array([  1.0, 0.5, 0.0, 0.0, 
                               0.0, 1.0, 0.0, 0.0, 
                               0.0, 0.0, 1.0, 0.0, 
                               0.0, 0.0, 0.0, 1.0], np.float32)
    
    
    mat_final = mult(mat_scale, mat_rotation)
    mat_final = mult(mat_final, mat_translation)
    mat_final = mult(mat_final, mat_shearing)
    
    loc = glGetUniformLocation(program, "mat_transformation")
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_final)
    
    glDrawArrays(GL_TRIANGLES, 0, 3)

    
    glfw.swap_buffers(window)

glfw.terminate()